In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
data = pd.read_csv('data_final.csv')
# Keeping only the neccessary columns
data = data[['Headline','Price movement']]

In [3]:
data = data[data['Price movement'] != "#N/A"]
data['Headline'] = data['Headline'].apply(lambda x: x.lower())
data['Headline'] = data['Headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[data['Price movement'] == 'Up'].size)
print(data[data['Price movement'] == 'Down'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Headline'].values)
X = tokenizer.texts_to_sequences(data['Headline'].values)
X = pad_sequences(X)

4490
3724


In [4]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           256000    
                                                                 
 conv1d (Conv1D)             (None, 32, 128)           65664     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 64)            32832     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 8, 64)            0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 8, 32)             8

In [5]:
Y = pd.get_dummies(data['Price movement']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2844, 32) (2844, 2)
(1402, 32) (1402, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, verbose = 2)

Epoch 1/20
89/89 - 1s - loss: 2.6653 - accuracy: 0.5313 - 1s/epoch - 17ms/step
Epoch 2/20
89/89 - 1s - loss: 1382.7449 - accuracy: 0.5018 - 657ms/epoch - 7ms/step
Epoch 3/20
89/89 - 1s - loss: 84300.6328 - accuracy: 0.4993 - 707ms/epoch - 8ms/step
Epoch 4/20
89/89 - 1s - loss: 466003.4375 - accuracy: 0.5186 - 672ms/epoch - 8ms/step
Epoch 5/20
89/89 - 1s - loss: 2099275.0000 - accuracy: 0.5207 - 679ms/epoch - 8ms/step
Epoch 6/20
89/89 - 1s - loss: 7469342.0000 - accuracy: 0.4993 - 735ms/epoch - 8ms/step
Epoch 7/20
89/89 - 1s - loss: 20727156.0000 - accuracy: 0.5232 - 955ms/epoch - 11ms/step
Epoch 8/20
89/89 - 1s - loss: 38001304.0000 - accuracy: 0.5042 - 1s/epoch - 14ms/step
Epoch 9/20
89/89 - 1s - loss: 113051792.0000 - accuracy: 0.5021 - 1s/epoch - 13ms/step
Epoch 10/20
89/89 - 1s - loss: 126347056.0000 - accuracy: 0.5025 - 1s/epoch - 12ms/step
Epoch 11/20
89/89 - 1s - loss: 243778304.0000 - accuracy: 0.5155 - 971ms/epoch - 11ms/step
Epoch 12/20
89/89 - 1s - loss: 451234016.0000 - acc

In [7]:
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

44/44 - 1s - loss: 5893816320.0000 - accuracy: 0.4879 - 820ms/epoch - 19ms/step
Accuracy: 48.79%
